In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
import re
import json


In [ ]:
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=851ecab1-0622-4dbe-a6c7-f950cf82abf9&limit=30000'  
with urllib.request.urlopen(url) as response:
     data = response.read()#.decode('utf-8')
     json_data = json.loads(data)
     records = json_data['result']['records']
     df = pd.DataFrame(records)

In [47]:
df.tail()

,_id,mispar_rechev,tozeret_cd,tozeret_nm,degem_cd,degem_nm,sug_rechev_cd,sug_rechev_nm,moed_aliya_lakvish,bitul_dt,...,ramat_eivzur_betihuty,kvutzat_zihum,shnat_yitzur,baalut,tzeva_rechev,zmig_kidmi,zmig_ahori,sug_delek_nm,horaat_rishum,kinuy_mishari
29995,29996,2810175,751,שברולט ד.קוריא,0761,PJ5E,112,פרטי נוסעים,2011,2024-02-13,...,0,15,2011,פרטי,אפור מטל,205/60R16,205/60R16,בנזין,0,CRUZE 1.6 LS
29996,29997,7107774,885,קיה קוריאה,0860,GE2262,112,פרטי נוסעים,2011,2024-02-13,...,0,15,2011,פרטי,ברונזה,205/60R16,205/60R16,דיזל,0,MAGENTIS
29997,29998,8555868,731,פורד-בלגיה,0201,BA7,112,פרטי נוסעים,2010,2024-02-13,...,0,15,2010,פרטי,כסף,215/55R16,215/55R16,בנזין,0,MONDEO
29998,29999,7288078,845,יונדאי טורקיה,0001,BB51C,112,פרטי נוסעים,2012,2024-02-13,...,0,14,2012,פרטי,בז מטאלי,175/70R14,175/70R14,בנזין,0,I20
29999,30000,5097675,730,פורד גרמניה,0006,DYB,112,פרטי נוסעים,2012,2024-02-13,...,0,15,2012,פרטי,שחור מטלי,205/55R16,205/55R16,בנזין,0,FOCUS


In [24]:
df.shape

(32000, 21)

In [34]:
df.dtypes

_id                       int64
mispar_rechev             int64
tozeret_cd                int64
tozeret_nm               object
degem_cd                 object
degem_nm                 object
sug_rechev_cd             int64
sug_rechev_nm            object
moed_aliya_lakvish       object
bitul_dt                 object
misgeret                 object
tozar_manoa              object
degem_manoa              object
mispar_manoa             object
mishkal_kolel             int64
ramat_gimur              object
ramat_eivzur_betihuty     int64
kvutzat_zihum             int64
shnat_yitzur              int64
baalut                   object
tzeva_rechev             object
zmig_kidmi               object
zmig_ahori               object
sug_delek_nm             object
horaat_rishum             int64
kinuy_mishari            object
dtype: object

In [33]:
df.isnull().sum().sort_values()

_id                      0
sug_delek_nm             0
zmig_ahori               0
zmig_kidmi               0
tzeva_rechev             0
baalut                   0
shnat_yitzur             0
kvutzat_zihum            0
ramat_eivzur_betihuty    0
ramat_gimur              0
mishkal_kolel            0
mispar_manoa             0
degem_manoa              0
tozar_manoa              0
misgeret                 0
bitul_dt                 0
moed_aliya_lakvish       0
sug_rechev_nm            0
sug_rechev_cd            0
degem_nm                 0
degem_cd                 0
tozeret_nm               0
tozeret_cd               0
mispar_rechev            0
horaat_rishum            0
kinuy_mishari            0
dtype: int64

### columns summary:

1. `mispar_rechev`: Numeric column representing the vehicle's number.
2. `tozeret_cd`: Numeric column representing a code associated with the vehicle's manufacturer.
3. `tozeret_nm`: Text column representing the name of the vehicle's manufacturer.
4. `degem_cd`: Text column representing a code associated with the vehicle's model.
5. `degem_nm`: Text column representing the name of the vehicle's model.
6. `sug_rechev_cd`: Numeric column representing a code associated with the vehicle's type.
7. `sug_rechev_nm`: Text column representing the name of the vehicle's type.
8. `moed_aliya_lakvish`: Text column representing the date of vehicle's import.
9. `bitul_dt`: Text column representing the date of vehicle's cancellation.
10. `misgeret`: Text column representing the vehicle's classification.
11. `tozar_manoa`: Text column representing the manufacturer's code.
12. `degem_manoa`: Text column representing the model's code.
13. `mispar_manoa`: Text column representing the vehicle's number within a model.
14. `mishkal_kolel`: Numeric column representing the vehicle's weight.
15. `ramat_gimur`: Text column representing the vehicle's assembly status.
16. `ramat_eivzur_betihuty`: Numeric column representing the vehicle's insured value.
17. `kvutzat_zihum`: Numeric column representing the assembly group.
18. `shnat_yitzur`: Numeric column representing the production year.
19. `baalut`: Text column representing the vehicle's condition.
20. `tzeva_rechev`: Text column representing the vehicle's color.
21. `zmig_kidmi`: Text column representing the internal fuel system.
22. `zmig_ahori`: Text column representing the external fuel system.
23. `sug_delek_nm`: Text column representing the name of the fuel type.
24. `horaat_rishum`: Numeric column representing the registration status.
25. `kinuy_mishari`: Text column representing the vehicle's registration mark.